In [7]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt
from math import cos,sin
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from keras.layers import Dense, Activation
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
pd.set_option('display.max_columns', 100)
import warnings
warnings.filterwarnings('ignore')
import pickle
from sklearn.metrics import mean_absolute_error
import keras_tuner as kt
import gc


In [2]:
def x_and_y(row,lat,lon):
    if lat[-1] == "N":
        lat = float(lat[:-1])
    else:
        lat = -float(lat[:-1])
    if lon[-1] == "E":
        lon = float(lon[:-1])
    else:
        lon = -float(lon[:-1])
    row['Latitude'] = lat
    row['Longitude'] = lon
    row['x'] = cos(lat) * cos(lon)
    row['y'] = cos(lat) * sin(lon)
    row['z'] = sin(lat)
    return row

In [3]:
df = pd.read_csv("GlobalLandTemperaturesByCity.csv")
df['x'] = 0
df['y'] = 0
df['z'] = 0
cities = df.groupby('City').max()[['Latitude','Longitude','x','y','z']].apply(lambda row: x_and_y(row,row["Latitude"],row["Longitude"]),axis=1).reset_index()
df = df[['dt','AverageTemperature','AverageTemperatureUncertainty','City','Country']].merge(cities,on='City')
df = df.astype({'dt':'datetime64'})
df['Month'] = df['dt'].dt.month
df['Year'] = df['dt'].dt.year
df = df.astype({'City':str,'Latitude':'float32','Longitude':'float32','x':'float32','y':'float32','z':'float32','Month':'int32','Year':'int32'})
df

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,x,y,z,Month,Year
0,1743-11-01,6.068,1.737,Århus,Denmark,57.049999,10.33,-0.541522,-0.689770,0.480594,11,1743
1,1743-12-01,NaN,NaN,Århus,Denmark,57.049999,10.33,-0.541522,-0.689770,0.480594,12,1743
2,1744-01-01,NaN,NaN,Århus,Denmark,57.049999,10.33,-0.541522,-0.689770,0.480594,1,1744
3,1744-02-01,NaN,NaN,Århus,Denmark,57.049999,10.33,-0.541522,-0.689770,0.480594,2,1744
4,1744-03-01,NaN,NaN,Århus,Denmark,57.049999,10.33,-0.541522,-0.689770,0.480594,3,1744
...,...,...,...,...,...,...,...,...,...,...,...,...
8599207,2013-05-01,11.464,0.236,Zwolle,Netherlands,52.240002,5.26,-0.204533,0.335398,0.919605,5,2013
8599208,2013-06-01,15.043,0.261,Zwolle,Netherlands,52.240002,5.26,-0.204533,0.335398,0.919605,6,2013
8599209,2013-07-01,18.775,0.193,Zwolle,Netherlands,52.240002,5.26,-0.204533,0.335398,0.919605,7,2013
8599210,2013-08-01,18.025,0.298,Zwolle,Netherlands,52.240002,5.26,-0.204533,0.335398,0.919605,8,2013


In [4]:
model_df = pd.DataFrame()
for city in df.City.unique():
    mydf = df.query("City == '"+city+"'").reset_index()
    mydf['TempMinus1'] = mydf.shift(12)['AverageTemperature']
    mydf['TempMinus2'] = mydf.shift(24)['AverageTemperature']
    model_df = pd.concat([model_df,mydf])
model_df

,index,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,x,y,z,Month,Year,TempMinus1,TempMinus2
0,0,1743-11-01,6.068,1.737,Århus,Denmark,57.049999,10.33,-0.541522,-0.689770,0.480594,11,1743,NaN,NaN
1,1,1743-12-01,NaN,NaN,Århus,Denmark,57.049999,10.33,-0.541522,-0.689770,0.480594,12,1743,NaN,NaN
2,2,1744-01-01,NaN,NaN,Århus,Denmark,57.049999,10.33,-0.541522,-0.689770,0.480594,1,1744,NaN,NaN
3,3,1744-02-01,NaN,NaN,Århus,Denmark,57.049999,10.33,-0.541522,-0.689770,0.480594,2,1744,NaN,NaN
4,4,1744-03-01,NaN,NaN,Århus,Denmark,57.049999,10.33,-0.541522,-0.689770,0.480594,3,1744,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3234,8599207,2013-05-01,11.464,0.236,Zwolle,Netherlands,52.240002,5.26,-0.204533,0.335398,0.919605,5,2013,14.169,14.066
3235,8599208,2013-06-01,15.043,0.261,Zwolle,Netherlands,52.240002,5.26,-0.204533,0.335398,0.919605,6,2013,14.702,16.273
3236,8599209,2013-07-01,18.775,0.193,Zwolle,Netherlands,52.240002,5.26,-0.204533,0.335398,0.919605,7,2013,17.329,16.020
3237,8599210,2013-08-01,18.025,0.298,Zwolle,Netherlands,52.240002,5.26,-0.204533,0.335398,0.919605,8,2013,18.588,16.975


In [9]:
del(df)
gc.collect()

237

In [10]:
model_df2 = pd.DataFrame()
for city in model_df.City.unique():
    mydf = model_df.query("City == '"+city+"'").reset_index()
    edf = pd.DataFrame()
    edf['dt'] = list(pd.date_range('2013-10-01','2060-01-01',freq='M'))
    edf['City'] = mydf.loc[0,'City']
    edf['Country'] = mydf.loc[0,'Country']
    edf['Latitude'] = mydf.loc[0,'Latitude']
    edf['Longitude'] = mydf.loc[0,'Longitude']
    edf['x'] = mydf.loc[0,'x']
    edf['y'] = mydf.loc[0,'y']
    edf['z'] = mydf.loc[0,'z']
    edf['Month'] = edf['dt'].dt.month
    edf['Year'] = edf['dt'].dt.year
    
    model_df2 = pd.concat([model_df2,mydf,edf])
model_df2


,level_0,index,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,x,y,z,Month,Year,TempMinus1,TempMinus2
0,0.0,0.0,1743-11-01,6.068,1.737,Århus,Denmark,57.049999,10.33,-0.541522,-0.689770,0.480594,11,1743,NaN,NaN
1,1.0,1.0,1743-12-01,NaN,NaN,Århus,Denmark,57.049999,10.33,-0.541522,-0.689770,0.480594,12,1743,NaN,NaN
2,2.0,2.0,1744-01-01,NaN,NaN,Århus,Denmark,57.049999,10.33,-0.541522,-0.689770,0.480594,1,1744,NaN,NaN
3,3.0,3.0,1744-02-01,NaN,NaN,Århus,Denmark,57.049999,10.33,-0.541522,-0.689770,0.480594,2,1744,NaN,NaN
4,4.0,4.0,1744-03-01,NaN,NaN,Århus,Denmark,57.049999,10.33,-0.541522,-0.689770,0.480594,3,1744,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,NaN,NaN,2059-08-31,NaN,NaN,Zwolle,Netherlands,52.240002,5.26,-0.204533,0.335398,0.919605,8,2059,NaN,NaN
551,NaN,NaN,2059-09-30,NaN,NaN,Zwolle,Netherlands,52.240002,5.26,-0.204533,0.335398,0.919605,9,2059,NaN,NaN
552,NaN,NaN,2059-10-31,NaN,NaN,Zwolle,Netherlands,52.240002,5.26,-0.204533,0.335398,0.919605,10,2059,NaN,NaN
553,NaN,NaN,2059-11-30,NaN,NaN,Zwolle,Netherlands,52.240002,5.26,-0.204533,0.335398,0.919605,11,2059,NaN,NaN


In [11]:
model_df2.to_csv('cities_minustemps_expanded.csv',compression="gzip")